# 문장간 유사도 검사

In [3]:
!pip install -U scikit-learn
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 53.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 46.8 MB/s eta 0:00:00


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
import konlpy
from konlpy.tag import Okt
t = Okt()

vectorizer = TfidfVectorizer(min_df = 1, decode_error = 'ignore')

def sentence_token(contents):
  contents_tokens = [t.morphs(row) for row in contents]

  contents_for_vectorize = []
  for content in contents_tokens:
    sentence = ''
    for word in content:
      sentence = sentence + ' ' + word
    contents_for_vectorize.append(sentence)

  #tf-idf 벡터화
  X = vectorizer.fit_transform(contents_for_vectorize)
  num_samples, num_features = X.shape
  print(num_samples, num_features)
  return X


In [5]:
def new_token(new_post):
  new_post_tokens = [t.morphs(row) for row in new_post]
  new_post_for_vectorize = []
  for content in new_post_tokens:
    sentence = ''
    for word in content:
      sentence = sentence + ' ' + word
    new_post_for_vectorize.append(sentence)
  new_post_vec = vectorizer.transform(new_post_for_vectorize)
  print(new_post_for_vectorize)
  return new_post_vec

In [6]:
import scipy as sp
def dist_raw(v1,v2):
  delta = v1 - v2
  return sp.linalg.norm(delta.toarray())

In [7]:
def check_distance(X, new_post_vec, contents):
  best_doc = None
  best_dist = 65535
  best_i = None
  result = []
  for i in range(len(contents)):
    post_vec = X.getrow(i)
    d = dist_raw(post_vec, new_post_vec)
    #print(d)
    print('== Post %i with dist = %.2f : %s' %(i,d, contents[i]))
    if d < best_dist:
      best_dist = d
      best_i = i
      result = []
    if d == best_dist:
      result.append(contents[i])
  return best_i, best_dist, result

In [8]:
contents = [
    "작년 대비 소비자물가가 33% 증가했다.",
    "작년 대비 소비자물가가 33%증가했다.",
    "작년 대비 소비자물가가 15% 증가했다.",
    "내 나이는 올해 33살이다.",
    "문방구가서 펜 33개만 사와줄래?",
    "소비자물가가 크게 오를 전망이다.",
    "작년에 비해 소비자물가가 증가했다.",
    "작년에 비해 소비자물가가 33% 증가했다."
]
new_post = ["작년 대비 소비자물가가 33% 증가했다."]

X = sentence_token(contents)
new_post_vec = new_token(new_post)
best_i, best_dist, result = check_distance(X, new_post_vec, contents)
print("Best post is %i, dist=%.2f" %(best_i, best_dist))
print('-->', new_post)
print('---->', contents[best_i])

8 19
[' 작년 대비 소비자 물가 가 33% 증가 했다 .']
== Post 0 with dist = 0.00 : 작년 대비 소비자물가가 33% 증가했다.
== Post 1 with dist = 0.00 : 작년 대비 소비자물가가 33%증가했다.
== Post 2 with dist = 0.70 : 작년 대비 소비자물가가 15% 증가했다.
== Post 3 with dist = 1.33 : 내 나이는 올해 33살이다.
== Post 4 with dist = 1.33 : 문방구가서 펜 33개만 사와줄래?
== Post 5 with dist = 1.30 : 소비자물가가 크게 오를 전망이다.
== Post 6 with dist = 0.83 : 작년에 비해 소비자물가가 증가했다.
== Post 7 with dist = 0.72 : 작년에 비해 소비자물가가 33% 증가했다.
Best post is 0, dist=0.00
--> ['작년 대비 소비자물가가 33% 증가했다.']
----> 작년 대비 소비자물가가 33% 증가했다.
